# <center>Project Demo Description</center>

In [1]:
# Installs geemap package
import subprocess

try:
        import geemap                  # GEE Python API
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

try:
        import numpy as np
        import pandas as pd
        import matplotlib.pyplot as plt
        import seaborn as sns
        #importing machine learning packages 
        from sklearn import ensemble
        from sklearn.model_selection import train_test_split
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

except ImportError:
    print('Install error packages')

%matplotlib inline
# Checks whether this notebook is running on Google Colab
try:
        import google.colab               # Accessing notebook on colab
        import geemap.foliumap as emap    # Importing folium viz framework
        from geemap import ml             # Importing ML model
except:
        import geemap as emap

# Authenticates and initializes Earth Engine
import ee                                 # Importing Google Earth Engine Environment to python 

try:
        ee.Initialize()                    # Initialize GEE Env
except Exception as e: 
        ee.Authenticate()
        ee.Initialize()



In [2]:
# defining the area of intrest to work
#make sure to include the shapefile which is our AOI 
AOI=ee.FeatureCollection("projects/ee-{your_username_on_Google_earth_engine}/assets/Garissa1County")


In [3]:
from geemap import ml 

In [4]:
#Read CSV
#the data is acquired from google earth engine dataset having 0 is maize , 1 is sorghum and 2 cowpeas
url = "data.csv"
df = pd.read_csv(url)
df.head()

,B1,B10,B11,B2,B3,B4,B5,B6,B7,landcover
0,0.122302,294.31340,289.91718,0.096028,0.073693,0.050018,0.326577,0.158818,0.064839,0
1,0.127584,294.92730,290.86554,0.100121,0.085449,0.055149,0.380837,0.175744,0.068880,0
2,0.123550,294.46100,290.24756,0.095798,0.072986,0.047406,0.343034,0.144637,0.054489,0
3,0.120793,294.50427,290.22095,0.093911,0.072469,0.047838,0.332497,0.156121,0.061805,0
4,0.121784,294.69790,290.72357,0.095399,0.077658,0.058008,0.313814,0.165054,0.072416,0


In [5]:
df.shape

(900, 10)

In [6]:
df['landcover']#landcover 0 is maize , 1 is sorghum and 2 cowpeas

0      0
1      0
2      0
3      0
4      0
      ..
895    2
896    2
897    2
898    2
899    2
Name: landcover, Length: 900, dtype: int64

In [7]:
df.columns

Index(['B1', 'B10', 'B11', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'landcover'], dtype='object')

In [8]:
# specify the names of the features (i.e. band names) and label
# feature names used to extract out features and define what bands

feature_names = ['B1','B10','B11', 'B2', 'B3','B4', 'B5', 'B6', 'B7']
label = "landcover"

In [9]:
# get the features and labels into separate variables
X = df[feature_names]
y = df[label]

In [10]:
# Split features and target into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [11]:
# Instantiate and fit the RandomForestClassifier
forest = RandomForestClassifier()

forest.fit(X_train, y_train)

RandomForestClassifier()

In [12]:
# Make predictions for the test set
y_pred_test = forest.predict(X_test)

In [13]:
# View confusion matrix for test data and predictions
confusion_matrix(y_test, y_pred_test)

array([[75,  0,  0],
       [ 0, 75,  0],
       [ 0,  0, 75]])

In [14]:
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### We now need to convert the estimator into a string representation so that Earth Engine can interpret.

In [15]:

## convert the estimator into a list of strings
trees =  ml.rf_to_strings(forest,feature_names)

In [16]:
# create a ee classifier to use with ee objects from the trees
ee_classifier = ml.strings_to_classifier(trees)

In [17]:
# Make a cloud-free Landsat 8 composite (from raw imagery).
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')\
.filterBounds(AOI)\
.map(lambda image: image.clip(AOI));

image = ee.Algorithms.Landsat.simpleComposite(
  collection= l8.filterDate('2018-01-01', '2021-01-31'),
  asFloat= True
)

In [18]:
#this section we are selecting the bands we want to perform classification
# and then claasify using Random forest classifier
classified = image.select(feature_names).classify(ee_classifier)
# print(classified)

In [19]:
# display results
Map = geemap.Map()
Map.setCenter(37.577717495842506,0.3597340638009545,7)
# Map.addLayer(image,{"min":0.05, "max": 0.55}, 'image')
legend_keys = ['Maize', 'Sorghum', 'Cowpeas']
# colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
# legend_colors = ['red', 'green', 'blue']
legend_colors = [ (127, 255, 0),(255, 0, 0), (0, 0, 255)]

Map.add_legend(
    
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)

Map.addLayer(AOI, {}, 'Garissa')
Map.addLayer(classified, {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},'classification')

Map

Map(center=[0.3597340638009545, 37.577717495842506], controls=(WidgetControl(options=['position', 'transparent…